<a href="https://colab.research.google.com/github/SoumyadeepB/Reinforcement-Learning/blob/master/Cart_Pole_REINFORCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense

In [38]:
env = gym.make('CartPole-v1')

2

In [40]:
''' For a  Neural Network Model:
def policy_model(env):
    n_states = env.observation_space.shape[0]
    n_actions = env.action_space.n 
    model = Sequential()
    model.add(Dense(8, input_dim=n_states, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(n_actions, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001))

    return model'''


def policy(env, state, theta):
    """ TODO: return probabilities for actions under softmax action selection """
    p = np.dot(state,theta).flatten()
    
    policy = 1.0 / (1.0 + np.exp(-p)) #Sigmoid Function
    #print("Sigmoid:",policy)
    total = np.sum(policy)
    policy = policy/total
    #print(policy)
    
    return policy  

In [41]:
def discounted_rewards(rewards):
  gamma = 0.1
  G = np.zeros_like(rewards)
  future_rewards = 0
  for i in reversed(range(0,len(rewards))):
    future_rewards = gamma*future_rewards + rewards[i]
    G[i] = future_rewards    # G_t = R_(t+1) + γ * G_(t+1) 

  return G

In [42]:
def gradient(state,theta):
  # Using Finite differences
  h = 0.001
  theta1 = theta + h
 
  f0 = np.log( np.dot(state, theta ) )
  f1 = np.log( np.dot(state, theta1) )
  #print("diff: ",f1-f0)

  grad = (f1 - f0)/h

  return grad

In [43]:
def generate_episode(env, theta, display=False):
    """ Generates one episode and returns the list of states, the list of rewards and the list of actions of that episode """
    state = env.reset()
    states = [state]
    actions = []
    rewards = []
    for t in range(500):
        if display:
            env.render()
        p = policy(env, state, theta)
        #print("Policy: ",p)
        action = np.random.choice(len(p), p=p)

        state, reward, done, info = env.step(action)
        rewards.append(reward)
        actions.append(action)
        if done:
            break
        states.append(state)

    return states, rewards, actions




In [68]:
def REINFORCE(env):
    theta = np.random.rand(4, 2)  # policy parameters
    alpha = 0.01
    low = abs(env.observation_space.low)

    for e in range(10000):
        if e % 300 == 0:
            states, rewards, actions = generate_episode(env, theta, False)  # display the policy every 300 episodes
        else:
            states, rewards, actions = generate_episode(env, theta, False)

        T = len(states)
        print("episode: " + str(e) + " length: " + str(T))

        v = discounted_rewards(rewards)
        
        # TODO: keep track of previous 100 episode lengths and compute mean

        # TODO: implement the reinforce algorithm to improve the policy weights
        for t in range(T):
          state = states[t] + low
          
          print(state)          
          #print("Vt: ",v[t])
          grad = gradient(state,theta)
          #print(grad)

          theta +=  alpha * grad * v[t]

In [69]:
env = gym.make('CartPole-v1')
REINFORCE(env)
env.close()

Streaming output truncated to the last 5000 lines.
[4.75244262e+00 3.40282347e+38 4.33639688e-01 3.40282347e+38]
diff:  [8.26818676e-05 8.26548549e-05]
[4.76094028e+00 3.40282347e+38 4.24056553e-01 3.40282347e+38]
diff:  [8.26755876e-05 8.26485811e-05]
[4.77333614e+00 3.40282347e+38 4.08713530e-01 3.40282347e+38]
diff:  [8.26693091e-05 8.26423088e-05]
[4.78182816e+00 3.40282347e+38 3.99256657e-01 3.40282347e+38]
diff:  [8.26630320e-05 8.26360378e-05]
[4.79422545e+00 3.40282347e+38 3.83882394e-01 3.40282347e+38]
diff:  [8.26567564e-05 8.26297683e-05]
[4.80272582e+00 3.40282347e+38 3.74237027e-01 3.40282347e+38]
diff:  [8.26504822e-05 8.26235002e-05]
[4.80733396e+00 3.40282347e+38 3.70220675e-01 3.40282347e+38]
diff:  [8.26442094e-05 8.26172336e-05]
[4.81585673e+00 3.40282347e+38 3.60075834e-01 3.40282347e+38]
diff:  [8.26379380e-05 8.26109684e-05]
[4.82829494e+00 3.40282347e+38 3.43778768e-01 3.40282347e+38]
diff:  [8.26316681e-05 8.26047046e-05]
[4.84465068e+00 3.40282347e+38 3.2127002

KeyboardInterrupt: ignored

In [48]:
state= np.array([-0.03470816 ,-0.00487081 , 0.01776325 , 0.0267998 ])
theta = np.random.rand(4, 2)